In [1]:
import http.client
import json
import pandas as pd
import openpyxl

In [2]:
bookings_df = pd.read_excel("Hackathon_Middleware_Bookings.xlsx")

In [3]:
bookings_df

,Booking ID,Flight,From,To,Class,Name,Passenger ID,Segment ID
0,68BE5O,CX491,TPE,HKG,Y,SPENCE/LAWRENCE MR(ADT),510892B000014EBB,5009A2B00005D613
1,68EGEJ,CX491,TPE,HKG,Y,ASHLEY/MATTIE MISS(ADT),510812B00000C8DD,5009A2B00005E3D2
2,68JO8T,CX491,TPE,HKG,Y,WU/LONG MR(ADT),510892B000015B41,5009A2B00005F738
3,68DEVR,CX491,TPE,HKG,J,LI/WAI MAN MR(ADT),510892B00001515E,500A02B00006283D
4,NaN,CX491,TPE,HKG,J,WONG/KA WAI MISS(ADT),510892B000015206,500A02B00006283E
5,68EOJ6,CX259,HKG,LHR,Y,CHEUNG/TSZ WAH MR(ADT),510892B0000153AC,5009A2B00005E3F4
6,68FF2P,CX259,HKG,LHR,Y,LAM/KWOK FAI MR(ADT),510812B00000CADE,5009A2B00005E7EC
7,68K6BJ,CX259,HKG,LHR,J,ALLISON/CHLOE MS(ADT),510812B00000D1DE,500A02B000064390
8,68LIEJ,CX259,HKG,LHR,W,LEVY/MARVIN MR(ADT),510892B000015CB7,500A02B0000643B6
9,NaN,CX259,HKG,LHR,W,BENNETT/ELISABETH MS(ADT),510812B00000D2AF,500A02B0000643B7


In [4]:
flights_arr = []
for id in bookings_df.Flight:
    conn = http.client.HTTPSConnection("developers.cathaypacific.com")
    header = {"apiKey" : "EgawcPqu4SXp332msGAStryj7Dm50WOI"}
    conn.request("POST", "/hackathon-apigw/airport/flights?fid={}".format(id),headers=header)

    res = conn.getresponse()
    data = res.read()

    data_json = json.loads(data.decode("utf-8"))
    flights_arr = flights_arr + [data_json]

In [5]:
table = []
flight_dict = {}
for flight in flights_arr:
    date = str(flight['data']['scheduledDepartureDate'])
    flight_dict['scheduledDepartureDate'] = date[0:4] + '-' + date[4:6] + '-' + date[6:]
    times = flight['data']['flightPoints'][0]  
    flight_dict['std'] = times['departure']['timings'][0]['value']
    flight_dict['sta'] = times['arrival']['timings'][0]['value']
    flight_dict['id'] = flight['data']['flightDesignator']['carrierCode'] + str(flight['data']['flightDesignator']['flightNumber'])
    flight_dict['from'] = flight['included']['legs'][0]['boardPointIataCode']
    flight_dict['to'] = flight['included']['legs'][0]['offPointIataCode']
    flight_dict['aircraftType'] = flight['included']['legs'][0]['aircraftEquipment']['aircraftType']
    flight_dict['aircraftConfiguration'] = flight['included']['legs'][0]['aircraftEquipment']['aircraftConfiguration']['code']
    flight_dict['aircraftConfiguration'] = flight['included']['legs'][0]['aircraftEquipment']['aircraftConfiguration']['code']
    flight_dict['total_capacity'] = flight['included']['legs'][0]['legCounters']['totalCounters']['capacity']
    flight_dict['f_capacity'] = 0
    flight_dict['j_capacity'] = 0
    flight_dict['w_capacity'] = 0
    flight_dict['y_capacity'] = 0
    flight_dict['f_booked'] = 0
    flight_dict['j_booked'] = 0
    flight_dict['w_booked'] = 0
    flight_dict['y_booked'] = 0
    flight_dict['f_availability'] = 0
    flight_dict['j_availability'] = 0
    flight_dict['w_availability'] = 0
    flight_dict['y_availability'] = 0
    for item in flight['included']['legs'][0]['legCounters']['cabinCounters']:
        if item['cabinName'] == "F":
            flight_dict['f_capacity'] = item['counters']['capacity']
            flight_dict['f_booked'] = item['counters']['booked']
            flight_dict['f_availability'] = item['counters']['availability']
        if item['cabinName'] == "J":
            flight_dict['j_capacity'] = item['counters']['capacity']
            flight_dict['j_booked'] = item['counters']['booked']
            flight_dict['j_availability'] = item['counters']['availability']
        if item['cabinName'] == "W":
            flight_dict['w_capacity'] = item['counters']['capacity']
            flight_dict['w_booked'] = item['counters']['booked']
            flight_dict['w_availability'] = item['counters']['availability']
        if item['cabinName'] == "Y":
            flight_dict['y_capacity'] = item['counters']['capacity']
            flight_dict['y_booked'] = item['counters']['booked']
            flight_dict['y_availability'] = item['counters']['availability']
            

    table = table + [flight_dict]
    flight_dict = {}

In [6]:
df = pd.DataFrame(table)

In [7]:
#f - first
#j - business
#w - prim econ
#y - econ

In [8]:
df

,scheduledDepartureDate,std,sta,id,from,to,aircraftType,aircraftConfiguration,total_capacity,f_capacity,...,w_capacity,y_capacity,f_booked,j_booked,w_booked,y_booked,f_availability,j_availability,w_availability,y_availability
0,2023-11-19,2023-11-19T19:40:00,2023-11-19T21:45:00,CX491,TPE,HKG,333,33E-A01,262,0,...,0,223,0,6,0,8,0,33,0,215
1,2023-11-19,2023-11-19T19:40:00,2023-11-19T21:45:00,CX491,TPE,HKG,333,33E-A01,262,0,...,0,223,0,6,0,8,0,33,0,215
2,2023-11-19,2023-11-19T19:40:00,2023-11-19T21:45:00,CX491,TPE,HKG,333,33E-A01,262,0,...,0,223,0,6,0,8,0,33,0,215
3,2023-11-19,2023-11-19T19:40:00,2023-11-19T21:45:00,CX491,TPE,HKG,333,33E-A01,262,0,...,0,223,0,6,0,8,0,33,0,215
4,2023-11-19,2023-11-19T19:40:00,2023-11-19T21:45:00,CX491,TPE,HKG,333,33E-A01,262,0,...,0,223,0,6,0,8,0,33,0,215
5,2023-11-19,2023-11-19T23:55:00,2023-11-20T05:00:00,CX259,HKG,LHR,77W,77A-A01,294,6,...,34,201,0,4,9,14,6,49,25,187
6,2023-11-19,2023-11-19T23:55:00,2023-11-20T05:00:00,CX259,HKG,LHR,77W,77A-A01,294,6,...,34,201,0,4,9,14,6,49,25,187
7,2023-11-19,2023-11-19T23:55:00,2023-11-20T05:00:00,CX259,HKG,LHR,77W,77A-A01,294,6,...,34,201,0,4,9,14,6,49,25,187
8,2023-11-19,2023-11-19T23:55:00,2023-11-20T05:00:00,CX259,HKG,LHR,77W,77A-A01,294,6,...,34,201,0,4,9,14,6,49,25,187
9,2023-11-19,2023-11-19T23:55:00,2023-11-20T05:00:00,CX259,HKG,LHR,77W,77A-A01,294,6,...,34,201,0,4,9,14,6,49,25,187


In [9]:
df.to_excel("parsed_data/flight.xlsx")